In [3]:

import pandas as pd
import numpy as np
import altair as alt

!pip install geopandas
import geopandas as gpd

!pip install iso3166
from iso3166 import countries as iso3166_countries


from vega_datasets import data
alt.data_transformers.disable_max_rows()

from shapely.geometry import Polygon

## **Data Processing**
**Reading the Data**

In [10]:
medicine_file = 'Medicines.csv'
mortality_file = 'Mortality.csv'

df_med = pd.read_csv(medicine_file)
df_mort = pd.read_csv(mortality_file)

**Choosing Countries, Medicines, and Mortalities to focus on**

In [11]:
#List of all the countries in Europe to use for the worldmap
countries = ["Turkey", 'Germany', 'United Kingdom', 'France', 'Italy', 'Spain', 'Ukraine', 'Poland',
                 'Netherlands', 'Belgium',  'Greece', 'Portugal', 'Sweden', 'Hungary',  'Austria', 
                  'Switzerland', 'Denmark', 'Finland', 'Norway', 'Ireland',
                 'Lithuania']
                 
all_of_europe = ["Turkey", 'Germany', 'United Kingdom', 'France', 'Italy', 'Spain', 'Ukraine', 'Poland', 'Romania', 
                 'Netherlands', 'Belgium', 'Czechia', 'Greece', 'Portugal', 'Sweden', 'Hungary', 'Belarus', 'Austria', 
                 'Serbia', 'Switzerland', 'Bulgaria', 'Denmark', 'Finland', 'Slovakia', 'Norway', 'Ireland', 'Croatia', 
                 'Moldova', 'Bosnia and Herzegovina', 'Albania', 'Lithuania', 'North Macedonia', 'Slovenia', 'Latvia', 'Estonia', 
                 'Montenegro', 'Luxembourg', 'Malta']

years = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]

#Chose general pharmaceuticals rather than specific ones to give a broad overview. Makes it easier to connect the cause of mortality and the pharmaceutical
#product
medicine = ['Total pharmaceutical sales','A-Alimentary tract and metabolism','B-Blood and blood forming organs','C-Cardiovascular system',
            'G-Genito urinary system and sex hormones','H-Systemic hormonal preparations, excluding sex hormones and insulins',
            'J-Antiin"fectives for systemic use','N-Nervous system','R-Respiratory system']

#Connected the causes of mortality with the different pharmaceutical product 
mortality = ['All causes of death','Certain infectious and parasitic diseases','Diseases of the nervous system',
             'Diseases of the blood and blood-forming organs','Endocrine, nutritional and metabolic diseases','Diseases of the respiratory system',
             'Diseases of the circulatory system','Mental and behavioural disorders']

#Chose to limit the measure of pharmaceutical and mortality rate to make it more standardized 
med_measure = ['Defined daily dosage per 1 000 inhabitants per day']
mort_measure =['Deaths per 100 000 population (standardised rates)']

def reduce_dataframe(df, country_list, variable_list, measure_list):
  df = df[(df["Country"].isin(country_list)) & (df["Variable"].isin(variable_list)) & (df["Measure"].isin(measure_list)) & (df["Year"].isin(years))]
  return df


df_med = reduce_dataframe(df_med, countries, medicine, med_measure)
df_mort = reduce_dataframe(df_mort, countries, mortality, mort_measure)


df_med.sample(5)


,VAR,Variable,UNIT,Measure,COU,Country,YEA,Year,Value,Flag Codes,Flags
4234,PHARHORM,"H-Systemic hormonal preparations, excluding se...",NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,NOR,Norway,2017,2017,48.2,NaN,NaN
3761,PHARGENU,G-Genito urinary system and sex hormones,NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,SWE,Sweden,2017,2017,104.7,NaN,NaN
6744,PHARRESS,R-Respiratory system,NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,NLD,Netherlands,2012,2012,106.2,D,Difference in methodology
3617,PHARGENU,G-Genito urinary system and sex hormones,NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,DNK,Denmark,2013,2013,101.2,NaN,NaN
145,PHARALIM,A-Alimentary tract and metabolism,NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,NLD,Netherlands,2016,2016,305.8,D,Difference in methodology


**Appending necessary iso3166 information**

In [12]:
# Needed to get the correct iso3166 id from country
def create_ids_from_countries(df):
  country_replacements = {"United Kingdom":"United Kingdom of Great Britain and Northern Ireland",
                          "Czech Republic": "Czechia",
                          "Korea": "Korea, Republic of",
                          "Slovak Republic": "Slovakia",
                          "Moldova": "Moldova, Republic of"}
                          
  df = df.replace(country_replacements)
  df["id"] = df["Country"].map(lambda x: int(iso3166_countries.get(x).numeric))

  return df

df_med = create_ids_from_countries(df_med)
df_mort = create_ids_from_countries(df_mort)

df_med.sample(5)

,VAR,Variable,UNIT,Measure,COU,Country,YEA,Year,Value,Flag Codes,Flags,id
158,PHARALIM,A-Alimentary tract and metabolism,NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,NOR,Norway,2019,2019,263.1,NaN,NaN,578
1197,PHARBLOO,B-Blood and blood forming organs,NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,GBR,United Kingdom of Great Britain and Northern I...,2014,2014,193.4,D,Difference in methodology,826
4269,PHARHORM,"H-Systemic hormonal preparations, excluding se...",NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,ESP,Spain,2020,2020,45.9,P,Provisional value,724
59081,PHARCARV,C-Cardiovascular system,NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,LTU,Lithuania,2017,2017,454.7,NaN,NaN,440
4264,PHARHORM,"H-Systemic hormonal preparations, excluding se...",NBPYEANB,Defined daily dosage per 1 000 inhabitants per...,ESP,Spain,2015,2015,35.0,D,Difference in methodology,724


In [13]:
# Creating a separate dataframe of all European countries to generate the background map

df_europe = pd.DataFrame()
df_europe["Country"] = all_of_europe
df_europe["Encoding"] = 1
df_europe = create_ids_from_countries(df_europe)

In [14]:
# change polygon geometry data for worldmap so extra territories are gone
gdf = gpd.read_file(data.world_110m.url)
france_polygon = Polygon([(5.675456754567563, 49.52991879123999), (5.898658986589879, 49.44190574386673), (6.186661866618664, 49.463909005710036), (6.658266582665846, 49.201562422193575), (8.098280982809825, 49.01707353443041), (7.594275942759424, 48.33327985868428), (7.468274682746824, 47.620712686681486), (7.191071910719103, 47.44976426774497), (6.7374673746737415, 47.54116243232488), (6.76986769867699, 47.28727864182508), (6.0390603906039075, 46.725349185518866), (6.021060210602116, 46.273436038429196), (6.499864998649997, 46.42915142993576), (6.841868418684186, 45.990778751672764), (6.802268022680238, 45.7081214649163), (7.097470974709751, 45.33237345497662), (6.748267482674834, 45.02771290637685), (7.0074700747007626, 44.25421362465411), (7.551075510755112, 44.127271729404214), (7.435874358743604, 43.693976726951234), (6.528665286652881, 43.12866215343834), (4.555845558455587, 43.399471529971464), (3.101431014310151, 43.074500278131694), (2.986229862298643, 42.473641973948844), (1.8270182701827196, 42.34331496149228), (0.700207002070016, 42.79522810858194), (0.3366033660336711, 42.58027316595877), (-1.5030150301502943, 43.03387887165175), (-1.9026190261902514, 43.42316735041811), (-1.384213842138422, 44.022333095997624), (-1.1934119341193252, 46.01447457211941), (-2.2266222662226482, 47.063860906185255), (-2.9646296462964585, 47.56993592858154), (-4.491044910449091, 47.95414673153789), (-4.591845918459171, 48.683639489574006), (-3.2958329583295836, 48.90197954940382), (-1.6182161821618024, 48.644710641697344), (-1.9350193501934996, 49.77703234732647), (-0.9882098820988006, 49.34712246208014), (1.337413374133746, 50.127391978216195), (1.6398163981639868, 50.94659034222889), (2.5146251462514613, 51.148004816025406), (2.658626586265882, 50.79764518513568), (3.123031230312307, 50.780719599102355), (3.5874358743587607, 50.379583210112656), (4.285842858428595, 49.907359359783044), (4.80064800648006, 49.98521705553631), (5.675456754567563, 49.52991879123999)])
norway_polygon = Polygon([(31.10251102511026, 69.55796474446768), (29.399693996939988, 69.15682835547801), (28.593285932859345, 69.06543019089807), (29.01449014490146, 69.76614945267752), (27.7328773287733, 70.16390072446055), (26.181261812618146, 69.82538900379416), (25.68805688056881, 69.0925111285514), (24.73404734047341, 68.64906077447839), (23.66123661236614, 68.89109665475488), (22.358023580235823, 68.84201245525824), (21.245612456124576, 69.37009073949784), (20.644406444064458, 69.10605159737804), (20.02520025200252, 69.06543019089807), (19.877598775987764, 68.40702489420192), (17.994779947799486, 68.56781796151847), (17.728377283772858, 68.01096618102224), (16.76716767167673, 68.01435129822889), (16.108361083610845, 67.30178412622612), (15.10755107551077, 66.19315824104365), (13.55593555935559, 64.78664204167475), (13.919539195391962, 64.44474520380169), (13.570335703357046, 64.048686490622), (12.580325803258035, 64.0656120766553), (11.93231932319324, 63.12793461040937), (11.993519935199345, 61.80096866539708), (12.630726307263075, 61.29320108439748), (12.29952299522995, 60.11856541368505), (11.467914679146787, 59.43307917933561), (11.028710287102882, 58.855916695599376), (10.355503555035568, 59.4703154686089), (8.382683826838274, 58.31260538392979), (7.0470704707047105, 58.07903229666999), (5.6646566465664705, 58.58849243627293), (5.308253082530825, 59.66326714938876), (4.991449914499157, 61.97022452573029), (5.913059130591307, 62.615089353599785), (8.551885518855187, 63.45459842085245), (10.528305283052845, 64.48536661028166), (12.357123571235718, 65.88003489942722), (14.761947619476189, 67.81124426582906), (16.435964359643606, 68.56274028570846), (19.182791827918294, 69.8169262107775), (21.378813788137876, 70.25529888904046), (23.02403024030241, 70.20282957233718), (24.546845468454705, 71.03049072936653), (26.36846368463685, 70.98648420567991), (28.164881648816504, 71.18620612087307), (31.29331293312933, 70.45332824563033), (30.00450004500047, 70.18590398630386), (31.10251102511026, 69.55796474446768)])
gdf.at[55,'geometry'] = france_polygon
gdf.at[118,'geometry'] = norway_polygon

# Concat shape information and production information so we don't have to use a lookup transformation
df_med = df_med.merge(gdf.astype({'id': 'int64'}), how='inner', on='id')

df_med = gpd.GeoDataFrame(df_med)

df_mort = df_mort.merge(gdf.astype({'id': 'int64'}), how='inner', on='id')

df_mort = gpd.GeoDataFrame(df_mort)

df_med = df_med.replace({"United Kingdom of Great Britain and Northern Ireland":"United Kingdom"})
df_mort= df_mort.replace({"United Kingdom of Great Britain and Northern Ireland":"United Kingdom"})
df_europe = df_europe.replace({"United Kingdom of Great Britain and Northern Ireland":"United Kingdom"})

## **Creating the Visualization**
**Definding the selectors**

In [15]:
med_domain = df_med['Variable'].unique()
mort_domain = df_mort['Variable'].unique()

medicine_dropdown = alt.binding_select(options=med_domain, name="Filter by Medicine Type:")
medicine_select = alt.selection_single(fields=['Variable'], bind=medicine_dropdown, name="Filter by Medicine Types", 
                                       init ={'Variable':'A-Alimentary tract and metabolism'} )
                                      

mort_dropdown = alt.binding_select(options=mort_domain, name="Filter by Mortality Type:")
mort_select = alt.selection_single(fields=['Variable'], bind=mort_dropdown, name="Filter by Cause of Mortality",
                                   init = {'Variable':'Certain infectious and parasitic diseases'})
                                



# Defining year slider
slider = alt.binding_range(name = "Select Year:", min=2010, max=2020, step=1)
select_year = alt.selection_single(fields=['Year'], bind=slider, init={'Year': 2010})

# Defining country selecter for line chart
select_country = alt.selection_multi(fields = ["Country"])

**Creating the background world map**

In [16]:
dom = [1]
rng = ["gray"]


# Creating the background world map
worldmap = alt.Chart(gdf).mark_geoshape(
    stroke='black',
).encode(
    fill=alt.Fill('Encoding:Q', scale=alt.
                  Scale(domain=dom, range=rng), legend = None),
    tooltip=["Country:N"]
).transform_lookup(
    lookup = "id",
    from_=alt.LookupData(df_europe, key = "id", fields = ["Encoding", "Country"])
)

**Creating the Medicine display**

In [21]:
# Creating the choloropeth on top
choropleth_med = alt.Chart(df_med).mark_geoshape().encode(
    color=alt.condition(select_country, alt.Color("Value:Q", scale=alt.Scale(domain = [0,1000])), alt.value('lightgray'),title ='Defined daily dosage per 1,000 inhabitants per day'),
    tooltip = ["Country:N", "Value:Q", "Year:Q"],
).add_selection(
    select_year,
    select_country,
    medicine_select
).transform_filter(
    select_year & medicine_select
)


bar_med = alt.Chart(df_med).mark_bar(size = 10).encode(
    x=alt.X("Value:Q",scale =alt.Scale(domain = [0,1000]),title = 'Defined daily dosage per 1,000 inhabitants per day'),
    y=alt.Y("Country:N"),
    color=alt.condition(select_country, alt.Color('Country:N', scale=alt.Scale(scheme='dark2', domain = countries)), alt.value('lightgray'))
).transform_filter(
    select_year & medicine_select
).properties(
    width = 400,
    height = 300
)

# Creating the accompanying country line chart
line_chart_med = alt.Chart(df_med).mark_line().encode(
    x = alt.X("Year:Q", axis=alt.Axis(format='d')),
    y = alt.Y("Value:Q", title ='Defined daily dosage per 1,000 inhabitants per day'),
    color=alt.condition(select_country, alt.Color('Country:N', 
                        scale=alt.Scale(scheme='dark2', domain = countries)), 
                        alt.value('lightgray'))
).properties(
    title = "Medicines across Time",
    width = 400,
    height = 300
).add_selection(
    select_country,
    medicine_select
).transform_filter(
    medicine_select
)

year_line = alt.Chart(df_med).mark_rule().encode(
   x = alt.X("Year:Q", axis=alt.Axis(format='d')),
).transform_filter(
    select_year
)


# Create combined pharma display
med_display = (alt.layer(worldmap, choropleth_med).properties(
    title = "Medicines in Europe",
    width = 400,
    height = 600
).project('mercator')) | ((line_chart_med + year_line) & bar_med) 


**Creating the Mortality Display**

In [22]:
# Creating the choloropeth on top (mortality)
choropleth_mort = alt.Chart(df_mort).mark_geoshape().encode(
    color=alt.condition(select_country, alt.Color("Value:Q", scale=alt.Scale(domain = [0,100])), alt.value('lightgray'),title = 'Deaths per 100,000 population (standardised rates)'),
    tooltip = ["Country:N", "Value:Q", "Year:Q"],
).add_selection(
    select_year,
    select_country,
    medicine_select
).transform_filter(
    select_year & mort_select
)


bar_mort = alt.Chart(df_mort).mark_bar(size = 10).encode(
    x=alt.X("Value:Q",scale =alt.Scale(domain = [0,150]),title = 'Deaths per 100,000 population (standardised rates)'),
    y=alt.X("Country:N"),
    color=alt.condition(select_country, alt.Color('Country:N', scale=alt.Scale(scheme='dark2', domain = countries)), alt.value('lightgray'))
).transform_filter(
    select_year & mort_select
).properties(
    width = 400,
    height = 300
)



# Creating the accompanying country line chart (mortality)
line_chart_mort = alt.Chart(df_mort).mark_line().encode(
    x = alt.X("Year:Q", axis=alt.Axis(format='d'), scale=alt.Scale(domain = [2010, 2020])),
    y = alt.Y("Value:Q", title = 'Deaths per 100,000 population (standardised rates)'),
    color=alt.condition(select_country, alt.Color('Country:N', scale=alt.Scale(scheme='dark2', domain = countries)), alt.value('lightgray'))
).properties(
    title = "Mortality across Time",
    width = 400,
    height = 300
).add_selection(
    select_country,
    mort_select
).transform_filter(
    mort_select
)

#Create combined mortality display
mort_display = (alt.layer(worldmap, choropleth_mort).properties(
    title = "Mortalities in Europe",
    width = 400,
    height = 600
).project('mercator')) |  ((line_chart_mort + year_line) & bar_mort)

In [23]:
#Combine the two displays
(med_display) | mort_display

Output hidden; open in https://colab.research.google.com to view.

In [24]:
(med_display | mort_display).save("vizaceuticals.html")